<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/10-Adding_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables


In [1]:
!pip install -q llama-index==0.10.57 openai==1.37.0 llama-index-finetuning llama-index-postprocessor-cohere-rerank llama-index-embeddings-huggingface llama-index-embeddings-cohere cohere==5.6.2 tiktoken==0.7.0 chromadb==0.5.5  llama-index-vector-stores-chroma==0.1.10 llama-index-llms-gemini==0.1.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os

# Set the following API Keys in the Python environment. Will be used later.
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"
os.environ["GOOGLE_API_KEY"] = "<YOUR_API_KEY>"
os.environ["CO_API_KEY"] = "<YOUR_COHERE_KEY>"
cohere_key = os.environ["CO_API_KEY"]

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
# os.environ["GOOGLE_API_KEY"] = userdata.get('Google_api_key')
# cohere_key = userdata.get('CO_API_KEY')

In [3]:
# Allows running asyncio in environments with an existing event loop, like Jupyter notebooks.

import nest_asyncio

nest_asyncio.apply()

# Load a Model


In [4]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=1, model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

**Note: You can create a vector store from scratch using the code below, or you can load it from Hugging Face using the code provided in this notebook.**

### Create a VectoreStore


In [ ]:
import chromadb

# create client and a new collection
# chromadb.EphemeralClient saves data in-memory.
chroma_client = chromadb.PersistentClient(path="./ai_tutor_knowledge")
chroma_collection = chroma_client.create_collection("ai_tutor_knowledge")

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore

# Define a storage context object using the created vector database.
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

### Load the Dataset (JSONL)


### Download


In [ ]:
from huggingface_hub import hf_hub_download
file_path = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="ai_tutor_knowledge.jsonl",repo_type="dataset",local_dir="/content")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


ai_tutor_knowledge.jsonl:   0%|          | 0.00/6.96M [00:00<?, ?B/s]

### Read File


In [ ]:
import json
with open(file_path, "r") as file:
    ai_tutor_knowledge = [json.loads(line) for line in file]

len(ai_tutor_knowledge)

762

### Convert to Document obj


In [ ]:
from typing import List
from llama_index.core import Document

def create_docs_from_list(data_list: List[dict]) -> List[Document]:
    documents = []
    for data in data_list:
        documents.append(
            Document(
                doc_id=data["doc_id"],
                text=data["content"],
                metadata={  # type: ignore
                    "url": data["url"],
                    "title": data["name"],
                    "tokens": data["tokens"],
                    "source": data["source"],
                },
                excluded_llm_metadata_keys=[
                    "title",
                    "tokens",
                    "source",
                ],
                excluded_embed_metadata_keys=[
                    "url",
                    "tokens",
                    "source",
                ],
            )
        )
    return documents

doc = create_docs_from_list(ai_tutor_knowledge)

## Transforming


In [ ]:
from llama_index.core.text_splitter import TokenTextSplitter

# Define the splitter object that split the text into segments with 512 tokens,
# with a 128 overlap between the segments.
text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)

In [ ]:
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.ingestion import IngestionPipeline

# Create the pipeline to apply the transformation on each chunk,
# and store the transformed text in the chroma vector store.
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        QuestionsAnsweredExtractor(questions=3, llm=Settings.llm),
        SummaryExtractor(summaries=["prev", "self"], llm=Settings.llm),
        KeywordExtractor(keywords=10, llm=Settings.llm),
        OpenAIEmbedding(),
    ],
    vector_store=vector_store,
)

# Run the transformation pipeline.
nodes = pipeline.run(documents=doc, show_progress=True)

In [ ]:
# Compress the vector store directory to a zip file to be able to download and use later.
!zip -r vectorstore.zip ai_tutor_knowledge

# Load Indexes


**Note: If you created the vector store from scratch, please comment out the three code blocks/cells below.**


In [5]:
from huggingface_hub import hf_hub_download
vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore.zip",repo_type="dataset",local_dir="/content")

vectorstore.zip:   0%|          | 0.00/97.2M [00:00<?, ?B/s]

In [6]:
!unzip vectorstore.zip

Archive:  vectorstore.zip
   creating: ai_tutor_knowledge/
   creating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/length.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/index_metadata.pickle  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/link_lists.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/header.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/data_level0.bin  
  inflating: ai_tutor_knowledge/chroma.sqlite3  


In [7]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

# Load the vector store from the local storage.
db = chromadb.PersistentClient(path="./ai_tutor_knowledge")
chroma_collection = db.get_or_create_collection("ai_tutor_knowledge")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [8]:
from llama_index.core import VectorStoreIndex

# Create the index based on the vector store.
index = VectorStoreIndex.from_vector_store(vector_store)

# Query Dataset


In [9]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

# Define the Cohere Reranking object to return only the first two highest ranking chunks.

#cohere_rerank = CohereRerank(top_n=2, api_key=cohere_key)
cohere_rerank3 = CohereRerank(top_n=2, api_key=cohere_key,model = 'rerank-english-v3.0')

In [21]:
# Define the ServiceCotext object to tie the LLM for generating final answer,
# and the embedding model to help with retrieving related nodes.
# The `node_postprocessors` function will be applied to the retrieved nodes.
query_engine = index.as_query_engine(
    similarity_top_k=5, node_postprocessors=[cohere_rerank3]
)

res = query_engine.query("Write about Retrieval Augmented Generation?")

In [22]:
res.response

'Retrieval-Augmented Generation (RAG) is a method designed to enhance the performance of generative models, particularly large language models (LLMs). It addresses common challenges faced by LLMs, such as generating outdated information and fabricating facts. RAG achieves this by integrating pretraining techniques with retrieval-based models, allowing for a more reliable generation of responses.\n\nA typical RAG workflow involves several key processing steps: \n\n1. **Query Classification**: This initial step determines whether the retrieval of additional information is necessary for the given input query.\n   \n2. **Retrieval**: Relevant documents are efficiently obtained based on the query, utilizing external knowledge sources that can provide up-to-date information.\n\n3. **Reranking**: The retrieved documents are then refined in order based on their relevance, ensuring that the most pertinent information is prioritized.\n\n4. **Repacking**: This step organizes the relevant document

In [23]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 1c324686-fad7-4a41-bfd3-d44f9612ca91
Title	 Evaluation of Retrieval-Augmented Generation: A Survey:Research Paper Information
Text	 Authors: Hao Yu, Aoran Gan, Kai Zhang, Shiwei Tong, Qi Liu, and Zhaofeng Liu.Numerous studies of Retrieval-Augmented Generation (RAG) systems have emerged from various perspectives since the advent of Large Language Models (LLMs). The RAG system comprises two primary components: Retrieval and Generation. The retrieval component aims to extract relevant information from various external knowledge sources. It involves two main phases, indexing and searching. Indexing organizes documents to facilitate efficient retrieval, using either inverted indexes for sparse retrieval or dense vector encoding for dense retrieval. The searching component utilizes these indexes to fetch relevant documents based on the user's query, often incorporating the optional rerankers to refine the ranking of the retrieved documents. The generation component utilizes the retr

# Evaluate


In [13]:
# To develop a custom retriever with keyword index, we require access to all nodes. We use the index as a retriever and requesting it to fetch a
# large number of documents, we can ensure that the retriever returns every document stored in the vector store. (This method serves as a temporary
# solution because LlamaIndex currently lacks the capability to fetch all documents from a chromadb. However, this limitation may be addressed in
# future updates.)

# If you Generated Vector Store from Scratch Please Comment below code block
# Only Use this cell code blcok if you download the vector store from Hugging Face.


retriever = index.as_retriever(similarity_top_k=100000000)
# Retrieve all nodes
all_nodes = retriever.retrieve("Hello!")

nodes = [item.node for item in all_nodes]

# took only minimal nodes
nodes = nodes[5500:]

In [14]:
# try with only nodes
len(nodes)

334

In [16]:
from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.llms.gemini import Gemini

# Create questions for each segment. These questions will be used to
# assess whether the retriever can accurately identify and return the
# corresponding segment when queried.


rag_eval_dataset = generate_question_context_pairs(
    nodes, llm = Settings.llm, num_questions_per_chunk=1
)

# We can save the evaluation dataset as a json file for later use.
rag_eval_dataset.save_json("./rag_eval_dataset_rerank.json")

100%|██████████| 334/334 [05:37<00:00,  1.01s/it]


If you have uploaded the generated question JSON file, please uncomment the code in the next cell block. This will avoid the need to generate the questions manually, saving you time and effort.


In [ ]:
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

# Load the evaluation dataset from a json file.
rag_eval_dataset = EmbeddingQAFinetuneDataset.from_json(
    "./rag_eval_dataset_rerank.json"
)

### Cohere Rerank

In [17]:
import pandas as pd


#  A simple function to show the evaluation result.
def display_results_retriever(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

In [20]:
from llama_index.core.evaluation import RetrieverEvaluator

# We can evaluate the retievers with different top_k values.
for i in [2, 4, 6, 8, 10]:
    retriever = index.as_retriever(
        similarity_top_k=i, node_postprocessors=[cohere_rerank3]
    )
    retriever_evaluator = RetrieverEvaluator.from_metric_names(
        ["mrr", "hit_rate"], retriever=retriever
    )
    eval_results = await retriever_evaluator.aevaluate_dataset(rag_eval_dataset)
    print(display_results_retriever(f"Retriever top_{i}", eval_results))

    Retriever Name  Hit Rate       MRR
0  Retriever top_2  0.655689  0.585329
    Retriever Name  Hit Rate       MRR
0  Retriever top_4  0.745509  0.611776
    Retriever Name  Hit Rate      MRR
0  Retriever top_6  0.760479  0.61477
    Retriever Name  Hit Rate       MRR
0  Retriever top_8  0.763473  0.615198
     Retriever Name  Hit Rate       MRR
0  Retriever top_10  0.766467  0.612803
